## എംബെഡിംഗുകൾ

മുൻ ഉദാഹരണത്തിൽ, നാം `vocab_size` നീളമുള്ള ഉയർന്ന-ഡൈമെൻഷണൽ ബാഗ്-ഓഫ്-വേർഡ് വെക്ടറുകളിൽ പ്രവർത്തിച്ചിരുന്നു, കൂടാതെ കുറഞ്ഞ-ഡൈമെൻഷണൽ സ്ഥാനപരമായ പ്രതിനിധാന വെക്ടറുകളെ സ്പാർസ് വൺ-ഹോട്ട് പ്രതിനിധാനത്തിലേക്ക് വ്യക്തമായി മാറ്റിയിരുന്നു. ഈ വൺ-ഹോട്ട് പ്രതിനിധാനം മെമ്മറി കാര്യക്ഷമമല്ല. കൂടാതെ, ഓരോ വാക്കും പരസ്പരം സ്വതന്ത്രമായി പരിഗണിക്കപ്പെടുന്നു, അതിനാൽ വൺ-ഹോട്ട് എൻകോഡുചെയ്ത വെക്ടറുകൾ വാക്കുകൾക്കിടയിലെ അർത്ഥപരമായ സമാനതകൾ പ്രകടിപ്പിക്കുന്നില്ല.

ഈ യൂണിറ്റിൽ, നാം **News AG** ഡാറ്റാസെറ്റിനെ തുടർന്നു പരിശോധിക്കും. തുടങ്ങാൻ, ഡാറ്റ ലോഡ് ചെയ്ത് മുൻ യൂണിറ്റിൽ നിന്നുള്ള ചില നിർവചനങ്ങൾ നേടാം.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### എമ്പെഡ്ഡിംഗ് എന്താണ്?

**എമ്പെഡ്ഡിംഗ്** എന്ന ആശയം ഒരു വാക്കിന്റെ അർത്ഥപരമായ അർത്ഥം പ്രതിഫലിപ്പിക്കുന്ന കുറവ്-ഡൈമെൻഷണൽ സാന്ദ്ര വെക്ടറുകൾ ഉപയോഗിച്ച് വാക്കുകളെ പ്രതിനിധീകരിക്കുന്നതാണ്. നാം പിന്നീട് അർത്ഥപൂർണ്ണമായ വാക്ക് എമ്പെഡ്ഡിംഗുകൾ എങ്ങനെ നിർമ്മിക്കാമെന്ന് ചർച്ച ചെയ്യും, പക്ഷേ ഇപ്പോൾ എമ്പെഡ്ഡിംഗുകളെ ഒരു വാക്ക് വെക്ടറിന്റെ ഡൈമെൻഷണാലിറ്റി കുറയ്ക്കാനുള്ള ഒരു മാർഗമായി മാത്രം കരുതാം.

അതിനാൽ, ഒരു എമ്പെഡ്ഡിംഗ് ലെയർ ഒരു വാക്ക് ഇൻപുട്ടായി സ്വീകരിച്ച്, നിർദ്ദിഷ്ടമായ `embedding_size` ഉള്ള ഔട്ട്പുട്ട് വെക്ടർ ഉത്പാദിപ്പിക്കുന്നു. ഒരു അർത്ഥത്തിൽ, ഇത് ഒരു `Dense` ലെയറിനോട് വളരെ സമാനമാണ്, പക്ഷേ ഒരു വൺ-ഹോട്ട് എൻകോഡഡ് വെക്ടർ സ്വീകരിക്കുന്നതിന് പകരം, ഇത് ഒരു വാക്കിന്റെ നമ്പർ സ്വീകരിക്കാൻ കഴിയും.

നമ്മുടെ നെറ്റ്‌വർക്കിലെ ആദ്യ ലെയറായി ഒരു എമ്പെഡ്ഡിംഗ് ലെയർ ഉപയോഗിച്ച്, നാം ബാഗ്-ഓഫ്-വേർഡ്സ് മോഡലിൽ നിന്ന് **എമ്പെഡ്ഡിംഗ് ബാഗ്** മോഡലിലേക്ക് മാറാം, ഇവിടെ ആദ്യം നമ്മുടെ ടെക്സ്റ്റിലെ ഓരോ വാക്കും അനുയോജ്യമായ എമ്പെഡ്ഡിങ്ങിലേക്ക് മാറ്റുകയും, പിന്നീട് ആ എമ്പെഡ്ഡിങ്ങുകളുടെ മേൽ `sum`, `average` അല്ലെങ്കിൽ `max` പോലുള്ള ഒരു സംഗ്രഹ ഫംഗ്ഷൻ കണക്കാക്കുകയും ചെയ്യും.

![അഞ്ച് സീക്വൻസ് വാക്കുകൾക്കുള്ള ഒരു എമ്പെഡ്ഡിംഗ് ക്ലാസിഫയർ കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/embedding-classifier-example.b77f021a7ee67eee.webp)

നമ്മുടെ ക്ലാസിഫയർ ന്യൂറൽ നെറ്റ്‌വർക്ക് താഴെപ്പറയുന്ന ലെയറുകൾ ഉൾക്കൊള്ളുന്നു:

* `TextVectorization` ലെയർ, ഇത് ഒരു സ്ട്രിംഗ് ഇൻപുട്ടായി സ്വീകരിച്ച്, ടോക്കൺ നമ്പറുകളുടെ ടെൻസർ ഉത്പാദിപ്പിക്കുന്നു. നാം ചില യുക്തിപരമായ വാക്ക് സമാഹാര വലുപ്പം `vocab_size` നിർദ്ദേശിക്കുകയും, കുറവായി ഉപയോഗിക്കുന്ന വാക്കുകൾ അവഗണിക്കുകയും ചെയ്യും. ഇൻപുട്ട് ആകൃതി 1 ആയിരിക്കും, ഔട്ട്പുട്ട് ആകൃതി $n$ ആയിരിക്കും, കാരണം നാം $n$ ടോക്കണുകൾ ലഭിക്കും, ഓരോന്നിലും 0 മുതൽ `vocab_size` വരെ നമ്പറുകൾ അടങ്ങിയിരിക്കും.
* `Embedding` ലെയർ, ഇത് $n$ നമ്പറുകൾ സ്വീകരിച്ച്, ഓരോ നമ്പറിനെയും ഒരു നിശ്ചിത നീളമുള്ള (നമ്മുടെ ഉദാഹരണത്തിൽ 100) സാന്ദ്ര വെക്ടറിലേക്ക് കുറയ്ക്കുന്നു. അതിനാൽ, $n$ ആകൃതിയിലുള്ള ഇൻപുട്ട് ടെൻസർ $n \times 100$ ആകൃതിയിലുള്ള ടെൻസറായി മാറും.
* ആഗ്രീഗേഷൻ ലെയർ, ഇത് ഈ ടെൻസറിന്റെ ആദ്യ ആക്സിസിൽ ശരാശരി കണക്കാക്കുന്നു, അഥവാ വ്യത്യസ്ത വാക്കുകളുമായി ബന്ധപ്പെട്ട എല്ലാ $n$ ഇൻപുട്ട് ടെൻസറുകളുടെ ശരാശരി കണക്കാക്കും. ഈ ലെയർ നടപ്പിലാക്കാൻ, നാം `Lambda` ലെയർ ഉപയോഗിച്ച് ശരാശരി കണക്കാക്കുന്ന ഫംഗ്ഷൻ പാസ്സ് ചെയ്യും. ഔട്ട്പുട്ട് ആകൃതി 100 ആയിരിക്കും, ഇത് മുഴുവൻ ഇൻപുട്ട് സീക്വൻസിന്റെ സംഖ്യാത്മക പ്രതിനിധാനം ആയിരിക്കും.
* അന്തിമ `Dense` ലീനിയർ ക്ലാസിഫയർ.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` പ്രിന്റ് ഔട്ടിൽ, **output shape** കോളത്തിൽ, ആദ്യത്തെ ടെൻസർ ഡൈമെൻഷൻ `None` മിനിബാച്ച് സൈസിനാണ്, രണ്ടാമത്തേത് ടോക്കൺ സീക്വൻസിന്റെ നീളിനാണ്. മിനിബാച്ചിലെ എല്ലാ ടോക്കൺ സീക്വൻസുകൾക്കും വ്യത്യസ്ത നീളുകൾ ഉണ്ട്. ഇതിനെ എങ്ങനെ കൈകാര്യം ചെയ്യാമെന്ന് അടുത്ത ഭാഗത്ത് ചർച്ച ചെയ്യും.

ഇപ്പോൾ നെറ്റ്‌വർക്ക് ട്രെയിൻ ചെയ്യാം:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **കുറിപ്പ്** നാം ഡാറ്റയുടെ ഒരു ഉപസമൂഹത്തെ അടിസ്ഥാനമാക്കി വെക്ടറൈസർ നിർമ്മിക്കുകയാണ്. പ്രക്രിയ വേഗത്തിലാക്കുന്നതിനായി ഇത് ചെയ്യപ്പെടുന്നു, അതിനാൽ നമ്മുടെ ടെക്സ്റ്റിലെ എല്ലാ ടോക്കണുകളും വാക്കസഞ്ചയത്തിൽ ഉണ്ടാകാതിരിക്കാം. ഈ സാഹചര്യത്തിൽ, ആ ടോക്കണുകൾ അവഗണിക്കപ്പെടും, ഇത് അല്പം കുറവ് കൃത്യതയ്ക്ക് കാരണമാകാം. എന്നിരുന്നാലും, യാഥാർത്ഥ്യത്തിൽ ടെക്സ്റ്റിന്റെ ഒരു ഉപസമൂഹം സാധാരണയായി നല്ല വാക്കസഞ്ചയ അളവുകൂട്ടലിന് സഹായിക്കുന്നു.


### വ്യത്യസ്ത സീക്വൻസ് വലുപ്പങ്ങൾ കൈകാര്യം ചെയ്യൽ

മിനിബാച്ചുകളിൽ പരിശീലനം എങ്ങനെ നടക്കുന്നു എന്ന് നമുക്ക് മനസിലാക്കാം. മുകളിൽ നൽകിയ ഉദാഹരണത്തിൽ, ഇൻപുട്ട് ടെൻസറിന്റെ ഡൈമെൻഷൻ 1 ആണ്, കൂടാതെ 128-ലോംഗ് മിനിബാച്ചുകൾ ഉപയോഗിക്കുന്നു, അതിനാൽ ടെൻസറിന്റെ യഥാർത്ഥ വലുപ്പം $128 \times 1$ ആണ്. എന്നാൽ, ഓരോ വാചകത്തിലെ ടോക്കണുകളുടെ എണ്ണം വ്യത്യസ്തമാണ്. `TextVectorization` ലെയർ ഒരു സിംഗിൾ ഇൻപുട്ടിൽ പ്രയോഗിച്ചാൽ, ടോക്കണുകളുടെ എണ്ണം ടെക്സ്റ്റ് എങ്ങനെ ടോക്കൺ ചെയ്യപ്പെടുന്നു എന്നതിനെ ആശ്രയിച്ച് വ്യത്യസ്തമായിരിക്കും:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


എങ്കിലും, നാം വെക്ടറൈസർ പല സീക്വൻസുകളിലും പ്രയോഗിക്കുമ്പോൾ, അത് ഒരു ചതുരാകൃതിയിലുള്ള ടെൻസർ ഉത്പാദിപ്പിക്കണം, അതിനാൽ ഉപയോഗിക്കാത്ത ഘടകങ്ങൾ PAD ടോക്കൺ (നമ്മുടെ കേസിൽ ഇത് പൂജ്യം) ഉപയോഗിച്ച് പൂരിപ്പിക്കുന്നു:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ഇവിടെ നാം എംബെഡിംഗുകൾ കാണാം:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **കുറിപ്പ്**: പാഡിംഗിന്റെ അളവ് കുറയ്ക്കാൻ, ചില സാഹചര്യങ്ങളിൽ ഡാറ്റാസെറ്റിലെ എല്ലാ സീക്വൻസുകളും വർദ്ധിക്കുന്ന നീളത്തിന്റെ (അഥവാ, കൂടുതൽ കൃത്യമായി, ടോക്കണുകളുടെ എണ്ണം) ക്രമത്തിൽ ക്രമീകരിക്കുന്നത് ഉചിതമാണ്. ഇതിലൂടെ ഓരോ മിനിബാച്ചിലും സമാന നീളമുള്ള സീക്വൻസുകൾ ഉൾപ്പെടുന്നതായി ഉറപ്പാക്കാം.


## സിമാന്റിക് എംബെഡിംഗുകൾ: Word2Vec

മുൻ ഉദാഹരണത്തിൽ, എംബെഡിംഗ് ലെയർ വാക്കുകളെ വെക്ടർ പ്രതിനിധാനങ്ങളായി മാപ്പ് ചെയ്യാൻ പഠിച്ചു, എന്നാൽ ഈ പ്രതിനിധാനങ്ങൾക്ക് സിമാന്റിക് അർത്ഥം ഉണ്ടായിരുന്നില്ല. സമാനമായ വാക്കുകൾ അല്ലെങ്കിൽ പദസമാനാർത്ഥങ്ങൾ ചില വെക്ടർ ദൂരങ്ങളിൽ (ഉദാഹരണത്തിന് യൂക്ലിഡിയൻ ദൂരം) അടുത്തുള്ള വെക്ടറുകളായി പ്രതിനിധാനം ചെയ്യപ്പെടുന്ന ഒരു വെക്ടർ പ്രതിനിധാനം പഠിക്കാനാകും നല്ലത്.

അത് ചെയ്യാൻ, വലിയ ഒരു ടെക്സ്റ്റ് ശേഖരത്തിൽ നമ്മുടെ എംബെഡിംഗ് മോഡൽ പ്രീട്രെയിൻ ചെയ്യേണ്ടതുണ്ട്, [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) പോലുള്ള സാങ്കേതിക വിദ്യ ഉപയോഗിച്ച്. വാക്കുകളുടെ വിതരണ പ്രതിനിധാനം സൃഷ്ടിക്കാൻ ഉപയോഗിക്കുന്ന രണ്ട് പ്രധാന ആർക്കിടെക്ചറുകൾ ഇതാണ്:

 - **കണ്ടിന്യൂവസ് ബാഗ്-ഓഫ്-വേർഡ്സ്** (CBoW), ഇവിടെ മോഡൽ ചുറ്റുപാടിലുള്ള കോൺടെക്സ്റ്റിൽ നിന്നൊരു വാക്ക് പ്രവചിക്കാൻ പരിശീലിപ്പിക്കുന്നു. ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ നൽകിയാൽ, മോഡലിന്റെ ലക്ഷ്യം $(W_{-2},W_{-1},W_1,W_2)$-ൽ നിന്നു $W_0$ പ്രവചിക്കുക എന്നതാണ്.
 - **കണ്ടിന്യൂവസ് സ്കിപ്പ്-ഗ്രാം** CBoW-യുടെ വിപരീതമാണ്. മോഡൽ നിലവിലെ വാക്ക് പ്രവചിക്കാൻ ചുറ്റുപാടിലുള്ള കോൺടെക്സ്റ്റ് വാക്കുകളുടെ വിൻഡോ ഉപയോഗിക്കുന്നു.

CBoW വേഗത്തിലാണ്, സ്കിപ്പ്-ഗ്രാം മന്ദഗതിയിലാണ്, എന്നാൽ അപൂർവമായ വാക്കുകൾ പ്രതിനിധാനം ചെയ്യുന്നതിൽ മികച്ചതാണ്.

![വാക്കുകൾ വെക്ടറുകളായി മാറ്റാൻ CBoWയും സ്കിപ്പ്-ഗ്രാം ആൽഗോരിതങ്ങളും കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News ഡാറ്റാസെറ്റിൽ പ്രീട്രെയിൻ ചെയ്ത Word2Vec എംബെഡിംഗ് പരീക്ഷിക്കാൻ, **gensim** ലൈബ്രറി ഉപയോഗിക്കാം. താഴെ 'neural' എന്ന വാക്കിനോട് ഏറ്റവും സമാനമായ വാക്കുകൾ കാണിക്കുന്നു.

> **Note:** വാക്ക് വെക്ടറുകൾ ആദ്യമായി സൃഷ്ടിക്കുമ്പോൾ, ഡൗൺലോഡ് ചെയ്യാൻ കുറച്ച് സമയം എടുക്കാം!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


നാം വാക്കിൽ നിന്നുള്ള വെക്ടർ എംബെഡ്ഡിംഗ് കൂടി എടുക്കാം, ക്ലാസിഫിക്കേഷൻ മോഡൽ പരിശീലനത്തിൽ ഉപയോഗിക്കാൻ. എംബെഡ്ഡിംഗിന് 300 ഘടകങ്ങളുണ്ട്, പക്ഷേ ഇവിടെ വ്യക്തതയ്ക്കായി വെക്ടറിന്റെ ആദ്യ 20 ഘടകങ്ങൾ മാത്രമാണ് കാണിക്കുന്നത്:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

സെമാന്റിക് എംബെഡിംഗുകളുടെ വലിയ ഗുണം എന്തെന്നാൽ, സെമാന്റിക്സിന്റെ അടിസ്ഥാനത്തിൽ വെക്ടർ എൻകോഡിംഗ് മാനിപ്പുലേറ്റ് ചെയ്യാൻ കഴിയുന്നതാണ്. ഉദാഹരണത്തിന്, *king* എന്ന വാക്കിനും *woman* എന്ന വാക്കിനും όσο അടുത്തുള്ള വെക്ടർ പ്രതിനിധാനം ഉള്ള ഒരു വാക്ക് കണ്ടെത്താൻ, കൂടാതെ *man* എന്ന വാക്കിൽ നിന്ന് όσο ദൂരമുള്ള ഒരു വാക്ക് കണ്ടെത്താൻ ഞങ്ങൾ ചോദിക്കാം:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

മുകളിൽ നൽകിയ ഉദാഹരണം ചില ആന്തരിക GenSym മാജിക് ഉപയോഗിച്ചാണ്, പക്ഷേ അടിസ്ഥാന തർക്കം വളരെ ലളിതമാണ്. embeddings-ന്റെ ഒരു രസകരമായ കാര്യം എന്തെന്നാൽ, embedding വെക്ടറുകളിൽ സാധാരണ വെക്ടർ പ്രവർത്തനങ്ങൾ നടത്താൻ കഴിയും, അത് വാക്കുകളുടെ **അർത്ഥങ്ങളിൽ** പ്രവർത്തനങ്ങൾ പ്രതിഫലിപ്പിക്കും. മുകളിൽ നൽകിയ ഉദാഹരണം വെക്ടർ പ്രവർത്തനങ്ങളുടെ രൂപത്തിൽ这样可以表达：നാം **KING-MAN+WOMAN** എന്ന വെക്ടർ കണക്കാക്കുന്നു (പ്രവർത്തനങ്ങൾ `+` ഉം `-` ഉം ബന്ധപ്പെട്ട വാക്കുകളുടെ വെക്ടർ പ്രതിനിധാനങ്ങളിൽ നടത്തപ്പെടുന്നു), തുടർന്ന് ആ വെക്ടറിനോട് ഏറ്റവും അടുത്ത വാക്ക് നിഘണ്ടുവിൽ കണ്ടെത്തുന്നു:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: *man* and *woman* വെക്ടറുകളിൽ ചെറിയ കോഫിഷ്യന്റുകൾ ചേർക്കേണ്ടിവന്നു - അവ നീക്കംചെയ്താൽ എന്ത് സംഭവിക്കുന്നുവെന്ന് നോക്കൂ.

ഏറ്റവും അടുത്ത വെക്ടർ കണ്ടെത്താൻ, TensorFlow യന്ത്രം ഉപയോഗിച്ച് നമ്മുടെ വെക്ടറും വാക്കുകളുടെ വോകാബുലറിയിലുള്ള എല്ലാ വെക്ടറുകളും തമ്മിലുള്ള ദൂരങ്ങളുടെ വെക്ടർ കണക്കാക്കുന്നു, പിന്നീട് `argmin` ഉപയോഗിച്ച് ഏറ്റവും കുറഞ്ഞ വാക്കിന്റെ ഇൻഡക്സ് കണ്ടെത്തുന്നു.


Word2Vec വാക്കുകളുടെ അർത്ഥം പ്രകടിപ്പിക്കാൻ മികച്ച മാർഗമായിരുന്നാലും, അതിന് നിരവധി ദോഷങ്ങൾ ഉണ്ട്, അവയിൽ ചിലത് താഴെപ്പറയുന്നവയാണ്:

* CBoW ഉം skip-gram മോഡലുകളും **പ്രഡിക്ടീവ് എംബെഡിംഗുകൾ** ആണെന്നും അവയ്ക്ക് പ്രാദേശിക കോൺടെക്സ്റ്റ് മാത്രമേ പരിഗണിക്കുകയുള്ളൂ എന്നും. Word2Vec ആഗോള കോൺടെക്സ്റ്റിന്റെ പ്രയോജനം ഉപയോഗപ്പെടുത്തുന്നില്ല.
* Word2Vec വാക്കിന്റെ **മോർഫോളജി** പരിഗണിക്കുന്നില്ല, അതായത് വാക്കിന്റെ അർത്ഥം വാക്കിന്റെ വിവിധ ഭാഗങ്ങളിൽ, ഉദാഹരണത്തിന് റൂട്ട്, ആശ്രയിച്ചിരിക്കാം എന്ന സത്യം പരിഗണിക്കുന്നില്ല.

**FastText** രണ്ടാം പരിമിതിയെ മറികടക്കാൻ ശ്രമിക്കുന്നു, Word2Vec-നെ അടിസ്ഥാനമാക്കി ഓരോ വാക്കിനും അതിലെ അക്ഷര n-ഗ്രാമുകൾക്കും വെക്ടർ പ്രതിനിധാനങ്ങൾ പഠിക്കുന്നു. ഓരോ ട്രെയിനിംഗ് ഘട്ടത്തിലും ഈ പ്രതിനിധാനങ്ങളുടെ മൂല്യങ്ങൾ ശരാശരി എടുത്ത് ഒരു വെക്ടറായി മാറ്റുന്നു. ഇതു പ്രീട്രെയിനിംഗിൽ അധിക കണക്കുകൂട്ടലുകൾ കൂട്ടിച്ചേർക്കുന്നുണ്ടെങ്കിലും, വാക്ക് എംബെഡിംഗുകൾക്ക് ഉപവാക്ക് വിവരങ്ങൾ കോഡ് ചെയ്യാൻ സാധ്യമാക്കുന്നു.

മറ്റൊരു രീതി, **GloVe**, വാക്ക് കോൺടെക്സ്റ്റ് മാട്രിക്സ് ഫാക്ടറൈസേഷനിൽ അടിസ്ഥാനമാക്കിയുള്ള വ്യത്യസ്ത സമീപനം ഉപയോഗിക്കുന്നു. ആദ്യം, വ്യത്യസ്ത കോൺടെക്സ്റ്റുകളിൽ വാക്കുകളുടെ സംഭവനകളുടെ എണ്ണം എണ്ണുന്ന വലിയ ഒരു മാട്രിക്സ് നിർമ്മിക്കുന്നു, പിന്നീട് ഈ മാട്രിക്സ് കുറവ് ഡൈമെൻഷനുകളിൽ പ്രതിനിധാനം ചെയ്യാൻ ശ്രമിക്കുന്നു, പുനർനിർമ്മാണ നഷ്ടം കുറയ്ക്കുന്ന വിധത്തിൽ.

gensim ലൈബ്രറി ആ വാക്ക് എംബെഡിംഗുകൾ പിന്തുണയ്ക്കുന്നു, മുകളിൽ നൽകിയ മോഡൽ ലോഡിംഗ് കോഡ് മാറ്റി അവയുമായി പരീക്ഷണം നടത്താം.


## Keras-ൽ pretrained embeddings ഉപയോഗിക്കൽ

മുകളിൽ നൽകിയ ഉദാഹരണം മാറ്റി, embedding ലെയറിൽ semantic embeddings, ഉദാഹരണത്തിന് Word2Vec ഉപയോഗിച്ച് matrix മുൻകൂട്ടി പൂരിപ്പിക്കാം. pretrained embedding-ന്റെ vocabularyയും ടെക്സ്റ്റ് കോർപ്പസിന്റെ vocabularyയും സാധാരണയായി പൊരുത്തപ്പെടില്ല, അതിനാൽ ഒരു vocabulary തിരഞ്ഞെടുക്കേണ്ടതുണ്ട്. ഇവിടെ രണ്ട് സാധ്യതകൾ പരിശോധിക്കുന്നു: tokenizer vocabulary ഉപയോഗിക്കൽ, Word2Vec embeddings-ന്റെ vocabulary ഉപയോഗിക്കൽ.

### Tokenizer vocabulary ഉപയോഗിക്കൽ

Tokenizer vocabulary ഉപയോഗിക്കുമ്പോൾ, vocabularyയിലെ ചില വാക്കുകൾക്ക് Word2Vec embeddings ഉണ്ടാകും, ചിലത് ഇല്ലാതിരിക്കും. നമ്മുടെ vocabulary വലുപ്പം `vocab_size` ആണെന്നും Word2Vec embedding വെക്ടർ നീളം `embed_size` ആണെന്നും കരുതുമ്പോൾ, embedding ലെയർ ഒരു വെയ്റ്റ് മാട്രിക്സ് ആയി പ്രതിനിധീകരിക്കും, അതിന്റെ ആകൃതി `vocab_size`$\times$`embed_size` ആണ്. vocabulary വഴി കടന്നുപോകിയാണ് ഈ മാട്രിക്സ് പൂരിപ്പിക്കുന്നത്:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec വാക്ക് കോശത്തിൽ ഇല്ലാത്ത വാക്കുകൾക്ക്, നാം അവയെ പൂജ്യം വാല്യുക്കളായി വയ്ക്കാമോ, അല്ലെങ്കിൽ ഒരു യാദൃച്ഛിക വെക്ടർ സൃഷ്ടിക്കാമോ.

ഇപ്പോൾ നാം pretrained ഭാരങ്ങളോടുകൂടിയ ഒരു embedding ലെയർ നിർവചിക്കാം:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

ഇപ്പോൾ നമുക്ക് നമ്മുടെ മോഡൽ പരിശീലിപ്പിക്കാം.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **കുറിപ്പ്**: `Embedding` സൃഷ്ടിക്കുമ്പോൾ `trainable=False` എന്ന് നമുക്ക് സജ്ജമാക്കിയിട്ടുണ്ട്, അതായത് Embedding ലെയർ വീണ്ടും പരിശീലിപ്പിക്കുന്നില്ല. ഇതിന്റെ ഫലമായി കൃത്യത കുറവാകാം, പക്ഷേ പരിശീലനം വേഗത്തിലാക്കും.

### embedding വാക്ക് സമാഹാരം ഉപയോഗിക്കൽ

മുൻപത്തെ സമീപനത്തിലെ ഒരു പ്രശ്നം TextVectorization-ലും Embedding-ലും ഉപയോഗിക്കുന്ന വാക്ക് സമാഹാരങ്ങൾ വ്യത്യസ്തമാണെന്നതാണ്. ഈ പ്രശ്നം മറികടക്കാൻ, താഴെ പറയുന്ന പരിഹാരങ്ങളിൽ ഒന്നോ അതിലധികമോ ഉപയോഗിക്കാം:
* Word2Vec മോഡൽ നമ്മുടെ വാക്ക് സമാഹാരത്തിൽ വീണ്ടും പരിശീലിപ്പിക്കുക.
* pretrained Word2Vec മോഡലിൽ നിന്നുള്ള വാക്ക് സമാഹാരം ഉപയോഗിച്ച് നമ്മുടെ ഡാറ്റാസെറ്റ് ലോഡ് ചെയ്യുക. ഡാറ്റാസെറ്റ് ലോഡുചെയ്യുമ്പോൾ ഉപയോഗിക്കുന്ന വാക്ക് സമാഹാരം വ്യക്തമാക്കാം.

ഇരണ്ടാമത്തെ സമീപനം എളുപ്പമാണ്, അതിനാൽ അത് നടപ്പിലാക്കാം. ആദ്യം, Word2Vec embeddings-ൽ നിന്നുള്ള നിശ്ചിത വാക്ക് സമാഹാരം ഉപയോഗിച്ച് `TextVectorization` ലെയർ സൃഷ്ടിക്കാം:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensim വാക്ക് എംബെഡ്ഡിംഗ്സ് ലൈബ്രറിയിൽ ഒരു സൗകര്യപ്രദമായ ഫംഗ്ഷൻ ഉണ്ട്, `get_keras_embeddings`, ഇത് സ്വയം അനുയോജ്യമായ Keras എംബെഡ്ഡിംഗ്സ് ലെയർ സൃഷ്ടിക്കും.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


നാം ഉയർന്ന കൃത്യത കാണാത്തതിന്റെ ഒരു കാരണമാണ് നമ്മുടെ ഡാറ്റാസെറ്റിലെ ചില വാക്കുകൾ പ്രീട്രെയിൻ ചെയ്ത GloVe വാക്കസഞ്ചാരത്തിൽ ഇല്ലാതിരിക്കുക, അതിനാൽ അവ അടിസ്ഥാനപരമായി അവഗണിക്കപ്പെടുന്നു. ഇതിനെ മറികടക്കാൻ, നാം നമ്മുടെ ഡാറ്റാസെറ്റിനെ അടിസ്ഥാനമാക്കി നമ്മുടെ സ്വന്തം എംബെഡിംഗുകൾ പരിശീലിപ്പിക്കാം.


## സാന്ദർഭിക എംബെഡിംഗുകൾ

Word2Vec പോലുള്ള പരമ്പരാഗത പ്രീട്രെയിൻഡ് എംബെഡിംഗ് പ്രതിനിധാനങ്ങളുടെ ഒരു പ്രധാന പരിമിതിയാണ്, അവ ഒരു വാക്കിന്റെ ചില അർത്ഥങ്ങൾ പിടിച്ചുപറ്റാൻ കഴിയുന്നുവെങ്കിലും, വ്യത്യസ്ത അർത്ഥങ്ങൾ തമ്മിൽ വ്യത്യാസപ്പെടുത്താൻ കഴിയാത്തത്. ഇത് ഡൗൺസ്ട്രീം മോഡലുകളിൽ പ്രശ്നങ്ങൾ സൃഷ്ടിക്കാം.

ഉദാഹരണത്തിന്, 'play' എന്ന വാക്കിന് ഈ രണ്ട് വ്യത്യസ്ത വാക്യങ്ങളിൽ വ്യത്യസ്ത അർത്ഥങ്ങളുണ്ട്:
- ഞാൻ തിയേറ്ററിൽ ഒരു **play** കാണാൻ പോയി.
- ജോൺ തന്റെ സുഹൃത്തുക്കളുമായി **play** ചെയ്യാൻ ആഗ്രഹിക്കുന്നു.

നാം ചർച്ച ചെയ്ത പ്രീട്രെയിൻഡ് എംബെഡിംഗുകൾ 'play' എന്ന വാക്കിന്റെ രണ്ട് അർത്ഥങ്ങളും ഒരേ എംബെഡിംഗിൽ പ്രതിനിധാനം ചെയ്യുന്നു. ഈ പരിമിതിയെ മറികടക്കാൻ, വലിയ ടെക്സ്റ്റ് കോർപ്പസിൽ പരിശീലിപ്പിച്ച **ഭാഷാ മോഡലിന്റെ** അടിസ്ഥാനത്തിൽ എംബെഡിംഗുകൾ നിർമ്മിക്കേണ്ടതുണ്ട്, ഇത് വാക്കുകൾ വ്യത്യസ്ത സാന്ദർഭങ്ങളിൽ എങ്ങനെ ചേർക്കാമെന്ന് *അറിയുന്നു*. സാന്ദർഭിക എംബെഡിംഗുകൾ ചർച്ച ചെയ്യുന്നത് ഈ ട്യൂട്ടോറിയലിന്റെ പരിധിക്ക് പുറത്താണ്, പക്ഷേ അടുത്ത യൂണിറ്റിൽ ഭാഷാ മോഡലുകൾക്കുറിച്ച് സംസാരിക്കുമ്പോൾ അവയെക്കുറിച്ച് വീണ്ടും വരാം.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
